In [2]:
import keras
import re
import pandas as pd
import tensorflow as tf
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from pymystem3 import Mystem
from sklearn.model_selection import train_test_split
stemmer = Mystem()

Using TensorFlow backend.
/home/timur/.local/share/virtualenvs/kaggle-1wSJiHo3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/timur/.local/share/virtualenvs/kaggle-1wSJiHo3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/timur/.local/share/virtualenvs/kaggle-1wSJiHo3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np

In [3]:
filename = 'data/labeled.csv'
dataframe = pd.read_csv(filename)

In [4]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
comment    14412 non-null object
toxic      14412 non-null float64
dtypes: float64(1), object(1)
memory usage: 225.3+ KB


Preprocess messages in our dataset 

In [5]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[0-9a-z#+_]')
STOPWORDS = set(stopwords.words('russian'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = text.replace('x', '')
    text = " ".join(stemmer.lemmatize(word)[0] for word in text.split() if word not in STOPWORDS)
    return text

In [6]:
dataframe[['comment', 'toxic']][:20]

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
5,Для каких стан является эталоном современная с...,1.0
6,В шапке были ссылки на инфу по текущему фильму...,0.0
7,УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РА...,1.0
8,"Ебать тебя разносит, шизик.\n",1.0
9,"Обосрался, сиди обтекай\n",1.0


In [7]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

In [16]:
def generat():
    for comment in dataframe['comment']:
        yield clean_text(comment)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(generat())
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 31004 unique tokens.


In [27]:
stemmer = Mystem()
X = tokenizer.texts_to_sequences(generat())
X = keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (14412, 250)


In [28]:
print(X)

[[    0     0     0 ...    20   331  5337]
 [    0     0     0 ...   356  6052   459]
 [    0     0     0 ...     2  3959   656]
 ...
 [    0     0     0 ...  2426 12069  1411]
 [    0     0     0 ...  3789  6533  4905]
 [    0     0     0 ...     4  3590  7676]]


In [29]:
Y = pd.get_dummies(dataframe['toxic'], ).values
print(Y.shape)

(14412, 2)


In [30]:
print(Y)

[[0 1]
 [0 1]
 [0 1]
 ...
 [1 0]
 [0 1]
 [1 0]]


In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .1, random_state=69)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(12970, 250) (12970, 2)
(1442, 250) (1442, 2)


In [32]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D, LSTM

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

In [33]:
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

Train on 10376 samples, validate on 2594 samples
Epoch 1/5
10376/10376 [==============================] - 51s 5ms/step - loss: 0.4878 - accuracy: 0.7627 - val_loss: 0.3214 - val_accuracy: 0.8620
Epoch 2/5
10376/10376 [==============================] - 51s 5ms/step - loss: 0.2055 - accuracy: 0.9246 - val_loss: 0.3084 - val_accuracy: 0.8809
Epoch 3/5
10376/10376 [==============================] - 51s 5ms/step - loss: 0.0964 - accuracy: 0.9673 - val_loss: 0.3711 - val_accuracy: 0.8712
Epoch 4/5
10376/10376 [==============================] - 51s 5ms/step - loss: 0.0501 - accuracy: 0.9850 - val_loss: 0.4196 - val_accuracy: 0.8616
Epoch 5/5
10376/10376 [==============================] - 51s 5ms/step - loss: 0.0293 - accuracy: 0.9916 - val_loss: 0.4932 - val_accuracy: 0.8616
